# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [79]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import imageio
import cv2
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)
#tf.set_random_seed(30)

Download the datasize & unzip it

In [3]:
!pip install gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 1.2 MB/s eta 0:00:00a 0:00:01
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=888ba37c79335b7d7f3aa63762e323d0079885248d4880d0a0f6abc00df5d955
  Stored in directory: /home/.cache/pip/wheels/7b/7b/5d/656f46cd6889e4c93977be9586901d0adc1271b2d876c84c96
Successfully built gdown
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [4]:
import gdown
url = 'https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL'
output = 'Project_data.zip'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1ehyrYBQ5rbQQe6yL4XbLWe3FMvuVUGiL
To: /home/Project_data.zip
100%|██████████| 1.71G/1.71G [01:48<00:00, 15.7MB/s]


'Project_data.zip'

In [5]:
!unzip 'Project_data.zip'

Archive:  Project_data.zip
   creating: Project_data/
   creating: Project_data/train/
  inflating: Project_data/train.csv  
   creating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00012.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00014.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00016.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00018.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00020.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00022.png  
  inflating: Project_data/train/WIN_20180907_15_35_09_Pro_Right Swipe_new/WIN_20180907_15_35_09_Pro_00024.png  
  inflating: Pro

In this block, you read the folder names for training and validation. You also set the batch_size here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [4]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 5 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [67]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = list(range(0,30))#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,30,120,160,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    image = cv2.resize(image, (160, 120))
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [68]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 50 # choose the number of epochs
print ('# epochs =', num_epochs)
nbout = 5

# training sequences = 663
# validation sequences = 100
# epochs = 50


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [69]:
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
#from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
#from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

In [70]:
#write your model here
conv_model = keras.Sequential()
momentum = .9
shape = (120,160,3)
conv_model.add(Conv2D(64, (3,3), input_shape=shape,
        padding='same', activation='relu'))
conv_model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
conv_model.add(BatchNormalization(momentum=momentum))
    
conv_model.add(MaxPool2D())
    
conv_model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
conv_model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
conv_model.add(BatchNormalization(momentum=momentum))
    
conv_model.add(MaxPool2D())
    
conv_model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
conv_model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
conv_model.add(BatchNormalization(momentum=momentum))
    
conv_model.add(MaxPool2D())
    
conv_model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
conv_model.add(Conv2D(512, (3,3), padding='same', activation='relu'))
conv_model.add(BatchNormalization(momentum=momentum))
    
# flatten...
conv_model.add(GlobalMaxPool2D())

#GRU model
model = keras.Sequential()
shape = (30,120,160,3)
model.add(TimeDistributed(conv_model, input_shape=shape))

model.add(GRU(64))
# and finally, we make a decision network
model.add(Dense(1024, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(nbout, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [71]:
optimiser = keras.optimizers.Adam(0.001)#write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_6 (TimeDis  (None, 30, 512)          4689216   
 tributed)                                                       
                                                                 
 gru_6 (GRU)                 (None, 64)                110976    
                                                                 
 dense_30 (Dense)            (None, 1024)              66560     
                                                                 
 dropout_18 (Dropout)        (None, 1024)              0         
                                                                 
 dense_31 (Dense)            (None, 512)               524800    
                                                                 
 dropout_19 (Dropout)        (None, 512)               0         
                                                     

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [72]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [73]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

LR = tf.keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.5,
           patience=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [74]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [75]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_436/385748685.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  Project_data/train ; batch size = 5
Epoch 1/50
133/133 [==============================] - ETA: 0s - loss: 1.7049 - categorical_accuracy: 0.2000Source path =  Project_data/val ; batch size = 5

Epoch 00001: saving model to model_init_2022-06-0814_47_18.313576/model-00001-1.70491-0.20000-1.61019-0.23000.h5
133/133 [==============================] - 78s 571ms/step - loss: 1.7049 - categorical_accuracy: 0.2000 - val_loss: 1.6102 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/50
133/133 [==============================] - ETA: 0s - loss: 1.6564 - categorical_accuracy: 0.1880
Epoch 00002: saving model to model_init_2022-06-0814_47_18.313576/model-00002-1.65638-0.18797-1.59469-0.25000.h5
133/133 [==============================] - 76s 571ms/step - loss: 1.6564 - categorical_accuracy: 0.1880 - val_loss: 1.5947 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 3/50
133/133 [==============================] - ETA: 0s - loss: 1.6431 - categorical_accuracy: 0.1880
Epoch 00

KeyboardInterrupt: 